In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Ablation study of point cloud size

In [ ]:
x = np.load("test_results/num_points1.npy")
y = np.vstack([np.array(row) for row in x[9:17]])
size = x[0:8]

In [ ]:
extra = np.load("test_results/num_points2.npy")
y = np.vstack((y, extra))
size = np.append(size, 5120)

In [ ]:
y_flat = np.ndarray.flatten(y)

In [ ]:
size_flat = np.repeat(size,50)

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
fig1, ax1 = plt.subplots()
fig1.set_figheight(13)
fig1.set_figwidth(13)
ax1.scatter(size_flat, y_flat, s=130, alpha=0.17)

ax1.errorbar(size, np.mean(y, axis=1), np.std(y, axis=1), marker='o', c=(1,0,0), capsize=20)
ax1.legend(["Individual Trial Accuracy", "Mean and STD"],loc=7, prop={'size': 20})
# ax1.plot(size, np.median(y, axis=1), 'r')
# ax1.plot(size, np.mean(y, axis=1), 'g')
ax1.set_xlabel("Number of Points",  fontsize=23)
ax1.set_ylabel("Overall Accuracy", fontsize=23)
ax1.tick_params(labelsize=20)

ax1.set_xscale('log')
ax1.set_xticks([20, 40, 80,160,320,640,1280,2560,5120])
ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

In [ ]:
np.mean(x,axis=1)

In [ ]:
import open3d as od
import numpy as np
import scipy.signal

# Downsampling/fusng visualization

In [ ]:
num = 110

In [ ]:
def croppedValues(arr):
        crop = []
        arr_min = np.min(arr[:,2])
        hist = np.histogram(arr[:,2], 50, (arr_min,arr_min+0.5))
        print(hist)
        peaks = scipy.signal.find_peaks(hist[0], distance=5)


        if len(peaks[0]) >= 2: # two peaks, cut face/body, isolate hand
            peak1 = peaks[0][0]
            peak2 = peaks[0][1]
            min_z = min(hist[1][peak1+3], hist[1][peak2-3])
            print(min_z)
            print("case 1")

        elif len(peaks[0]) == 1: # one peak, isolate hand
            peak = peaks[0][0]
            min_z = hist[1][peak+3]
            print(min_z)
            print("case 2")


        else: # no peaks, um, try something else...
            min_z = hist[1][2] + 0.2
            print("case 3")

        for i, entry in enumerate(arr):
            if entry[2] <= (min_z):
                crop.append(i)

        if len(crop) < 320: # we accidentally cropped the whole thing
            min_z = min(arr[:,2]) + 0.15
            print("fuck")

            for i, entry in enumerate(arr):
                if entry[2] <= (min_z):
                    crop.append(i)
            if len(crop) < 320: 
                return list(range(0,len(arr.T[0])))
        return crop

In [ ]:
x = np.load("test/four/fourfinger1/npys/0right.npy")
# x[0] = -x[0]
x = x.T
x = x[croppedValues(x)]
pcd = od.PointCloud()
pcd.points = od.Vector3dVector(x)

# mesh_frame = od.create_mesh_coordinate_frame(size = 0.5, origin = [0,0,0])
od.draw_geometries([pcd])

In [ ]:
tran = np.load("base_transform_new.npy")
for row in tran:
    out = ""
    for el in row:
        out = out + str(el) + " "
    out = out[:-1]
    print(out)


In [ ]:
newest_tran = np.asarray([[-0.010311, -0.743751, -0.668511, 0.411904],[0.739154, 0.444590, -0.505998, 0.291180],[0.673578, -0.499305, 0.545095, 0.255718],[0.000000, 0.000000, 0.000000, 1.000000]])
np.save("base_transform_newest", newest_tran)

In [ ]:
x = np.load("train/ell/ell2/npys/100left.npy")
pcd = od.PointCloud()
pcd.points = od.Vector3dVector(x.T)

y = np.load("train/ell/ell2/npys/100right.npy")
pcd2 = od.PointCloud()
pcd2.points = od.Vector3dVector(y.T)
pcd2.transform(newest_tran)

od.draw_geometries([pcd2,pcd])

In [ ]:
x = np.load("train/bird/bird2/npys/" + str(num) + "leftreduced.npy")
pcd = od.PointCloud()
pcd.points = od.Vector3dVector(x.T)
od.draw_geometries([pcd])

In [ ]:
x = np.load("train/two/twofinger4/npys/" + str(num) + "rightreduced.npy")
pcd = od.PointCloud()
pcd.points = od.Vector3dVector(x.T)
od.draw_geometries([pcd])

In [ ]:
x = np.load("train/zero/open4/npys/" + str(num) + "fused.npy")
pcd = od.PointCloud()
pcd.points = od.Vector3dVector(x.T)
# pcd = od.voxel_down_sample(pcd, voxel_size = 0.0160)
# print(len(pcd.points))
od.draw_geometries([pcd])

# Network Ablation

In [ ]:
ab = np.load("test_results/ablation.npy")

In [ ]:
print("Max Achieved in 300 iterations:")
print("Original Network: " + str(np.max(ab[0])))
print("Reduced Calorie Network: " + str(np.max(ab[1])))
print("Dropout Network: " + str(np.max(ab[2])))
print("Selu Network: " + str(np.max(ab[3])))
print("Reduced Network + Dropout: " + str(np.max(ab[4])))


In [ ]:
print("Mean Achieved in 300 iterations:")
print("Original Network: " + str(np.mean(ab[0])))
print("Reduced Calorie Network: " + str(np.mean(ab[1])))
print("Dropout Network: " + str(np.mean(ab[2])))
print("Selu Network: " + str(np.mean(ab[3])))
print("Reduced Network + Dropout: " + str(np.mean(ab[4])))


In [ ]:
print("STD Achieved in 300 iterations:")
print("Original Network: " + str(np.std(ab[0])))
print("Reduced Calorie Network: " + str(np.std(ab[1])))
print("Dropout Network: " + str(np.std(ab[2])))
print("Selu Network: " + str(np.std(ab[3])))
print("Reduced Network + Dropout: " + str(np.std(ab[4])))


In [ ]:
x = np.repeat([0,1,2,3,4],300)

In [ ]:
fig1, ax1 = plt.subplots()
fig1.set_figheight(13)
fig1.set_figwidth(13)
ax1.scatter(x, ab.flatten(), s=130, alpha=0.17)
ax1.scatter([0,1,2,3,4], np.mean(ab, axis=1),s=200)
# ax1.legend(["Individual Trial Accuracy", "Mean and STD"],loc=7, prop={'size': 20})
# ax1.plot(size, np.median(y, axis=1), 'r')
# ax1.plot(size, np.mean(y, axis=1), 'g')
ax1.set_xlabel("System Type",  fontsize=23)
ax1.set_ylabel("Overall Accuracy", fontsize=23)
ax1.tick_params(labelsize=20)


# Augmentation

In [ ]:
ab = np.load("test_results/augmentation.npy")
x = np.repeat([0,0.01,0.05,0.1,0.25],100)￼

ab_fine = np.load("test_results/augmentation_fine.npy")
x_fine = np.repeat([0,0.01,0.02,0.03,0.04,0.05],25)

In [ ]:
fig1, ax1 = plt.subplots()
fig1.set_figheight(13)
fig1.set_figwidth(13)
ax1.scatter(x, ab.flatten(), s=130, alpha=0.17)
ax1.scatter([0,0.01,0.05,0.1,0.25], np.mean(ab, axis=1),s=200)

ax1.scatter(x_fine, ab_fine.flatten(), s=130, alpha=0.17, c='b')
ax1.scatter([0,0.01,0.02,0.03,0.04,0.05], np.mean(ab_fine, axis=1),s=200,c='r')
ax1.legend(["Individual Trial Accuracy", "Mean"],loc=7, prop={'size': 20})
# ax1.plot(size, np.median(y, axis=1), 'r')
# ax1.plot(size, np.mean(y, axis=1), 'g')
ax1.set_xlabel("Augmentation Factor",  fontsize=23)
ax1.set_ylabel("Overall Accuracy", fontsize=23)
ax1.tick_params(labelsize=20)


# Results

In [ ]:
acc = np.load("test_results/acc.npy")
mat = np.load("test_results/mat.npy")

In [ ]:
import numpy
numpy.set_printoptions(threshold=numpy.nan)
# Class Accuracies
print("Class ACC")
print(np.round(np.mean(mat/100, axis=2),4))
# Class STD
print("Class STD")
print(np.round(np.std(mat/100, axis=2),4))
# Overall Accuracies
print("Overall ACC")
print(np.round(np.mean(acc,axis=1),4))
# Overall STD
print("Overall STD")
print(np.round(np.std(acc,axis=1),4))

In [ ]:
amat = 100 - mat

In [ ]:
a = np.array(np.round(np.mean(acc,axis=1),4),dtype=np.str)
slash = np.array([["/" for i in range(1)] for i in range(9)],dtype=np.str)
b = np.array(np.round(np.std(acc,axis=1),4),dtype=np.str)

res = np.asarray(np.core.defchararray.add(np.core.defchararray.add(a,slash),b),dtype=str)
print(res)

In [ ]:
print("Class Energy")
for row in amat:
    x = row.flatten()/100
    print(np.round(np.sum(x**2),3))


# Confusion Matrix

In [ ]:
conf = np.load("test_results/confusion.npy")

In [ ]:
plt.mat(conf[0])

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(np.round(conf[0]/100, 1), index = ["OPEN", "T.UP", "T.DOWN", "TWO", "BIRD", "FRAME"],
                  columns = ["OPEN", "T.UP", "T.DOWN", "TWO", "BIRD", "FRAME"])
plt.figure(figsize = (10,10))
sn.heatmap(df_cm, annot=True)

In [ ]:
conf[0]